In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
!pip install implicit

  Using cached https://files.pythonhosted.org/packages/bc/07/c0121884722d16e2c5beeb815f6b84b41cbf22e738e4075f1475be2791bc/implicit-0.4.4.tar.gz
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-win_amd64.whl size=666700 sha256=96af44c34f6446688ddb8cfa0ed3c4b9a20f66a3869b671c61d20985ed6e0881
  Stored in directory: C:\Users\6145~1\AppData\Local\pip\Cache\wheels\bf\d4\ec\fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [53]:
def weighted_random_recommendation(items_weights, n=5):
   
    items = items_weights['item_id']
    weights = items_weights['weight'] 
    
    recs = np.random.choice(items, p=weights, size=n, replace=False)
    
    return recs.tolist()

In [54]:
%%time

popularity = data_test.groupby('item_id').size().reset_index(name='item_count')
popularity['weight_prep'] = np.log(popularity['item_count']) 
popularity['weight'] = popularity['weight_prep'] / popularity['weight_prep'].sum()
columns = ['item_id', 'weight']
items_weight = popularity[columns]

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weight, n=5))
result.head(2)

Wall time: 923 ms


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[896617, 823365, 985361, 15926712, 13008469]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[834491, 1023021, 931635, 12302114, 8090532]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [39]:
result = pd.read_csv('predictions_basic.csv')
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[12487931, 831536, 1034556, 5768936, 13008291]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[13115896, 1121603, 6766506, 12385185, 10456579]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[13382383, 1319807, 12425387, 828999, 12158036]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[1082185, 1029743, 6534178, 1127831, 995785]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[6546594, 6395922, 1034773, 964390, 2436947]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[863350, 904881, 1082955, 10355816, 12695956]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]"


In [40]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [15]:
import ast
 
# код для корректной загрузки датасета
result['actual'] = result['actual'].apply(lambda row: np.fromstring(row[1:-1],sep=' ',dtype=int))
for column_name in result.columns[2:]:
    result[column_name] = result[column_name].apply(ast.literal_eval)
 
# код для красивого отображения метрик
for column_name in result.columns[1:]:
    print('%s: %f' % (column_name, result.apply(lambda row: precision_at_k(row[column_name], row['actual']), axis=1).mean()))

actual: 1.000000
random_recommendation: 0.000588
popular_recommendation: 0.155240
itemitem: 0.136925
cosine: 0.132909
tfidf: 0.138981


C:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


own_purchases: 0.201917


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### Улучшение бейзлайнов

In [47]:
result_5000 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_5000.columns=['user_id', 'actual']
result_5000.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [48]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [49]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)
top_5000

,item_id,n_sold
55470,6534178,190227964
55430,6533889,15978434
55465,6534166,12439291
55576,6544236,2501949
43620,1404121,1562004
...,...,...
51914,5574045,120
10610,864048,119
7777,838842,119
26071,1004001,119


In [50]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [52]:
%%time

items = top_5000['item_id']

result_5000['random_recommendation_5000'] = result_5000['user_id'].apply(lambda x: random_recommendation(items, n=5))
result_5000.head(2)

Wall time: 427 ms


,user_id,actual,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12384431, 1046689, 953320, 822346, 12172355]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[980727, 869344, 10345155, 999563, 1019230]"


In [53]:
%%time

result_5000['popular_recommendation_5000'] = result_5000['user_id'].apply(lambda x: top_5000.head(5).item_id.tolist())
result_5000.head(2)

Wall time: 322 ms


,user_id,actual,random_recommendation_5000,popular_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12384431, 1046689, 953320, 822346, 12172355]","[6534178, 6533889, 6534166, 6544236, 1404121]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[980727, 869344, 10345155, 999563, 1019230]","[6534178, 6533889, 6534166, 6544236, 1404121]"


In [54]:
for column_name in result_5000.columns[1:]:
    print('%s: %f' % (column_name, result_5000.apply(lambda row: precision_at_k(row[column_name], row['actual']), axis=1).mean()))

actual: 1.000000
random_recommendation_5000: 0.004701
popular_recommendation_5000: 0.046131


In [ ]:
#random_recommendation улучшился, но popular_recommendation, взятый от топ по колчеству проданных товаров ухудшился.

### Улучшение ItemItemRecommender

In [55]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
top_5000

[6534178,
 6533889,
 6534166,
 6544236,
 1404121,
 397896,
 1426702,
 5703832,
 480014,
 5668996,
 6410462,
 420647,
 5747420,
 731106,
 5716076,
 707683,
 202291,
 1153346,
 1082185,
 5747233,
 6410464,
 545926,
 995242,
 2848087,
 1388206,
 1029743,
 5712216,
 5850988,
 1133018,
 1106523,
 1007195,
 981760,
 5845857,
 883404,
 1127831,
 2690723,
 866227,
 995785,
 860776,
 951590,
 5569230,
 908531,
 916122,
 826249,
 1098066,
 862349,
 1058997,
 1044078,
 904360,
 840361,
 923746,
 1126899,
 849843,
 961554,
 1005186,
 820165,
 1053690,
 844179,
 844165,
 1070820,
 1065593,
 834484,
 994928,
 833715,
 859075,
 1022003,
 1013321,
 938700,
 854852,
 986947,
 5569471,
 927191,
 1071939,
 1096036,
 1004906,
 986912,
 1080414,
 914190,
 908846,
 962229,
 8090521,
 1085604,
 911878,
 1092026,
 866211,
 1068719,
 878996,
 8090537,
 1081177,
 847270,
 929668,
 1024306,
 909894,
 907014,
 903325,
 910032,
 1095275,
 833025,
 862139,
 962568,
 953476,
 847982,
 999971,
 968215,
 1028816,
 976

In [56]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Елена\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [59]:
%%time

model_2 = ItemItemRecommender(K=2, num_threads=4) # K - кол-во билжайших соседей
model_5 = ItemItemRecommender(K=5, num_threads=4)
model_10 = ItemItemRecommender(K=10, num_threads=4)
model_15 = ItemItemRecommender(K=15, num_threads=4)

model_2.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)
model_5.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)
model_10.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)
model_15.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 5.08 s


In [58]:
result_itemitem = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_itemitem.columns=['user_id', 'actual']
result_itemitem.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [60]:
%%time

result_itemitem['itemitem_2'] = result_itemitem['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_2.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

result_itemitem['itemitem_5'] = result_itemitem['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_5.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

result_itemitem['itemitem_10'] = result_itemitem['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_10.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

result_itemitem['itemitem_15'] = result_itemitem['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_15.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

Wall time: 516 ms


In [61]:
for column_name in result_itemitem.columns[1:]:
    print('%s: %f' % (column_name, result_itemitem.apply(lambda row: precision_at_k(row[column_name], row['actual']), axis=1).mean()))

actual: 1.000000
itemitem_2: 0.192009
itemitem_5: 0.136925
itemitem_10: 0.150930
itemitem_15: 0.153183


In [ ]:
# лучше всего показала себя модель с двумя соседями.